In [ ]:
# ----------------------------------------------
# Anomaly Detection with Anomalib: Setup Script
# ----------------------------------------------

import os
import torch
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from pytorch_lightning import seed_everything, Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

from anomalib import TaskType
import anomalib
from anomalib.data import Folder, MVTecAD
from anomalib.models import (
    Patchcore, Padim, Stfpm, Draem, EfficientAd, Fastflow, Cflow
)
from anomalib.deploy.inferencers import TorchInferencer
from anomalib.utils.normalization import NormalizationMethod
from anomalib.utils.post_processing import superimpose_anomaly_map
from anomalib.data.utils import TestSplitMode
print("Anomalib version:", anomalib.__version__)
seed_everything(0)
torch.set_float32_matmul_precision("medium")

In [ ]:
# Initialize the datamodule using Folder-based dataset
datamodule = Folder(
    name="my_dataset",
    root=r"D:\Anomaly_detecion\hazelnut_toy",
    normal_dir="good",
    abnormal_dir="crack",  # only if you have it you can do it with good only
    normal_split_ratio=0.1,

    train_batch_size=16,
    eval_batch_size=16,
)

datamodule.setup()



In [ ]:
model = Padim(
    backbone="resnet18",  # Feature extraction backbone
    layers=["layer1", "layer2", "layer3"],  # Layers to extrxtrct
    pre_trained=True,  # Use pretrained weights
    n_features=100,  # Number of features
)



In [14]:
engine = Engine(
    max_epochs=1,  # PaDiM needs only one epoch
    accelerator="auto",  # Automatically detect GPU/CPU
    devices=1,  # Number of devices to use
)

# Train the model
engine.fit(
    model=model,
    datamodule=datamodule,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\sam\anaconda3\envs\anomalib-gpu\lib\site-packages\lightning\pytorch\core\optimizer.py:183: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name           | Type          | Params | Mode 
---------------------------------------------------------
0 | pre_processor  | PreProcessor  | 0      | train
1 | post_processor | PostProcessor | 0      | train
2 | evaluator      | Evaluator     | 0      | train
3 | model          | PadimModel    | 2.8 M  | train
---------------------------------------------------------
2.8 M     Trainable params
0         Non-trainable params
2.8 M     Total params
11.131    Total estimated model params size (MB)
19        Modules in train mode
69  

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [15]:
path_list = engine.trainer.checkpoint_callback.best_model_path.split('\\')
engine.export(model, ExportType.TORCH, export_root = "/".join(path_list[6:9]))

WindowsPath('v0/weights/lightning/weights/torch/model.pt')

In [16]:
engine.test(datamodule=datamodule, model=model)


The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: Evaluator, ImageVisualizer, PostProcessor, PreProcessor
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\sam\anaconda3\envs\anomalib-gpu\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing: |          | 0/? [00:00<?, ?it/s]

c:\Users\sam\anaconda3\envs\anomalib-gpu\lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric AUROC was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)
c:\Users\sam\anaconda3\envs\anomalib-gpu\lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: The ``compute`` method of metric F1Score was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │            1.0            │
│       image_F1Score       │            0.5            │
└───────────────────────────┴───────────────────────────┘

[{'image_AUROC': 1.0, 'image_F1Score': 0.5}]

In [21]:
import torch
from PIL import Image
from torchvision.transforms import Compose, ToTensor, Resize
from anomalib.deploy.inferencers import TorchInferencer
import numpy as np
import torchvision.transforms as transforms

# Load the image
image_path = r"D:\Anomaly_detecion\hazelnut_toy\good\15.jpg"
print("sam")
image = Image.open(image_path).convert("RGB")  # Convert image to RGB



# Initialize the inferencer
inferencer = TorchInferencer(r'v0/weights/lightning/weights/torch/model.pt')

# Perform inference
results = inferencer.predict(image)  # Unsqueeze to add batch dimension
results.pred_score ,results.pred_label,results



sam


(tensor([[0.]], device='cuda:0'),
 tensor([False], device='cuda:0'),
 ImageBatch(image=Image([[[[0.8078, 0.8039, 0.8039,  ..., 0.8157, 0.8235, 0.8000],
          [0.8078, 0.8157, 0.8157,  ..., 0.8118, 0.8039, 0.7961],
          [0.7961, 0.8078, 0.8000,  ..., 0.8118, 0.8196, 0.8157],
          ...,
          [0.8392, 0.8235, 0.8392,  ..., 0.8196, 0.8078, 0.8157],
          [0.8392, 0.8314, 0.8471,  ..., 0.8157, 0.8039, 0.7843],
          [0.8314, 0.8392, 0.8392,  ..., 0.8196, 0.8157, 0.8078]],
 
         [[0.7804, 0.7725, 0.7725,  ..., 0.7804, 0.7882, 0.7647],
          [0.7765, 0.7843, 0.7843,  ..., 0.7765, 0.7686, 0.7608],
          [0.7647, 0.7765, 0.7686,  ..., 0.7765, 0.7843, 0.7804],
          ...,
          [0.8157, 0.8000, 0.8157,  ..., 0.7882, 0.7765, 0.7843],
          [0.8157, 0.8078, 0.8235,  ..., 0.7843, 0.7804, 0.7608],
          [0.8078, 0.8157, 0.8157,  ..., 0.7961, 0.7922, 0.7843]],
 
         [[0.2314, 0.2353, 0.2353,  ..., 0.2667, 0.2745, 0.2510],
          [0.2392, 0

In [17]:
import torch
from PIL import Image
from anomalib.deploy.inferencers import TorchInferencer

# Load image
image = Image.open(r"D:\Anomaly_detecion\hazelnut_toy\crack\04.jpg").convert("RGB")

# Initialize inferencer
inferencer = TorchInferencer(path=r"v0/weights/lightning/weights/torch/model.pt")
print(inferencer)

# Perform inference
result = inferencer.predict(image)

# Get raw anomaly score
pred_score = result.pred_score.item()


# Custom reversed threshold logic
custom_threshold = 0.4
pred_label = 1 if pred_score >custom_threshold else 0  # 🔁 REVERSE
label_name = "Anomalous" if pred_label == 1 else "Normal"

# Print results
print(f"🧠 Raw Anomaly Score     : {pred_score:.4f}")
print(f"🎯 Threshold Used        : {custom_threshold} (Reversed Logic)")
print(f"🏷️  Predicted Class Index: {pred_label}")
print(f"🔖 Predicted Class Label : {label_name}")


🧠 Raw Anomaly Score     : 0.4988
🎯 Threshold Used        : 0.4 (Reversed Logic)
🏷️  Predicted Class Index: 1
🔖 Predicted Class Label : Anomalous
